In [1]:
from typing import Union, List
import numpy as np

a = 3
a.__add__(4)

7

In [2]:
a = np.array([2, 3, 1, 0])
a.__add__(4)

array([6, 7, 5, 4])

In [4]:
a + 4

array([6, 7, 5, 4])

In [ ]:
# Gradient Accumulation
Numberable = Union[float, int]

def ensure_number(num: Numberable) -> NumberWithGrad:
    if isinstance(num, NumberWithGrad):
        return num
    return NumberWithGrad(num)

class NumberWithGrad():
    def __init__(self, num: Numberable, depends_on: List[Numberable] = None, creation_op: str = ""):
        self.num = num
        self.grad = None
        self.depends_on = depends_on or []
        self.creation_op = creation_op

    def __add__(self, other: Numberable):
        return NumberWithGrad(self.num + ensure_number(other).num, depends_on=[self, ensure_number(other)], creation_op='add')

    def __mul__(self, other: Numberable = None):
        return NumberWithGrad(self.num * ensure_number(other).num, depends_on=[self, ensure_number(other)], creation_op='mul')

    def backward(self, backward_grad: Numberable = None):
        if backward_grad is None:
            self.grad = 1
        else:
            if self.grad is None:
                self.grad = backward_grad
            else:
                self.grad += backward_grad
        if self.creation_op == "add":
            self.depends_on[0].backward(self.grad)
            self.depends_on[1].backward(self.grad)
        else: # mul
            new = self.depends_on[1] * self.grad
            self.depends_on[0].backward(new.num)
            new = self.depends_on[0] * self.grad
            self.depends_on[1].backward(new.num)